In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 

full = pd.read_csv("full_data.csv")

In [2]:
import pickle 
with open("openai_proxy_partition.pkl", "rb") as f:
    partition = pickle.load(f)

In [3]:
easiness = np.zeros(len(full))
for class_num in tqdm(range(len(partition)), position=0):
    easiness[partition[class_num]] += np.diag(np.load(f"image-text-zero-cos-sim/{class_num}.npy"))

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:12<00:00, 78.65it/s] 


In [4]:
THRESHOLD = 0
raw_scores = np.load(f"easy+cross+ls+div-sim-selected-frac-ls-threshold=0.npy")

In [5]:
itc_sim = np.zeros(len(full))

for class_num in tqdm(range(len(partition))):
    itc_sim[partition[class_num]] += np.load(f"image-text-center-zero-cos-sim/{class_num}.npy")

100%|██████████| 1000/1000 [00:00<00:00, 2034.53it/s]


In [6]:
tic_sim = np.zeros(len(full))

for class_num in tqdm(range(len(partition))):
    tic_sim[partition[class_num]] += np.load(f"text-image-center-zero-cos-sim/{class_num}.npy")

100%|██████████| 1000/1000 [00:00<00:00, 1904.56it/s]


In [7]:
label_sim = np.load("openai_proxy_text_cos_sim.npy")

In [8]:
inverse_partition = {}

for lc in range(1000):
    for i in partition[lc]:
        inverse_partition[i] = lc 

In [9]:
renorm_ls = np.zeros(len(full))

for i in tqdm(range(len(partition))):
    renorm_ls[partition[i]] += label_sim[partition[i], i]

100%|██████████| 1000/1000 [00:00<00:00, 2223.35it/s]


In [10]:
it_sims = {}
for i in tqdm(range(1000), position=0):
    it_sims[i] = np.load(f"image-text-zero-cos-sim/{i}.npy")
    if THRESHOLD is not None:
        it_sims[i] *= (it_sims[i] > THRESHOLD)
    it_sims[i] = it_sims[i].astype("float32")

 56%|█████▌    | 560/1000 [01:51<07:08,  1.03it/s]

In [ ]:
def cross_to_subset(class_num, subset):
    convert_subset = []  
    for i, idx in enumerate(partition[class_num]):
        if idx in subset:
            convert_subset.append(i)
    return ((np.sum(it_sims[class_num][:, convert_subset], axis=1) + np.sum(it_sims[class_num][convert_subset, :], axis=0))) / len(partition[class_num])

In [ ]:
def ls_of_subset(class_num, subset):
    convert_subset = []  
    for i, idx in enumerate(partition[class_num]):
        if idx in subset:
            convert_subset.append(i)
    return (np.sum(renorm_ls[subset])/2) / len(partition[class_num])

In [ ]:
subset_size = int(len(full) * 1 /10)
scores = (raw_scores[:, 0] + 2*easiness + renorm_ls/2) - (tic_sim + itc_sim) 
actual_scores = scores.copy()

In [20]:
scores = actual_scores.copy()
subset = []
counter = {}
old_counter = {}
selected = {}
new_selected = {}
for i in range(1000):
    counter[i] = 0
    old_counter[i] = 0
    new_selected[i] = []
    selected[i] = []
    
sorted_idx = np.argsort(scores)
pbar = tqdm(total=subset_size, position=0)
sort_count = 0

while len(sorted_idx) > 0 and len(subset) < subset_size:
    subset.append(sorted_idx[-1])
    curr_partition = inverse_partition[subset[-1]]
    selected[curr_partition].append(subset[-1])
    new_selected[curr_partition].append(subset[-1])
    counter[curr_partition] += 1
    pbar.set_description_str(f"score={scores[sorted_idx[-1]]}")
    pbar.update(1)

    if (counter[curr_partition] + 1) % 200 == 0:
        # deduct cross
        sim_selected = cross_to_subset(curr_partition, new_selected[curr_partition])
        scores[partition[curr_partition]] -= sim_selected
        new_cross_deduction = (np.sum(raw_scores[selected[curr_partition], 0]))  / (len(partition[curr_partition]))
        scores[partition[curr_partition]] -= new_cross_deduction
        
        # deduct ls of selected 
        ls_deduction = ls_of_subset(curr_partition, new_selected[curr_partition])
        scores[partition[curr_partition]] -= ls_deduction
        
        # Re-sort 
        scores[subset] = -np.inf
        sorted_idx = np.argsort(scores)[len(subset):]
        sort_count += 1
        old_counter[curr_partition] = counter[curr_partition]
        pbar.set_postfix_str(f"sort_count={sort_count}")
        
        # Reset new selected
        new_selected[curr_partition] = []
    else:
        sorted_idx = sorted_idx[:-1]

score=1.0438493142062784: 100%|█████████▉| 283780/283781 [09:20<00:00, 443.07it/s, sort_count=1250, sim-selected=0.014297999441623688, new_cross_deduction: 0.04337480014620724, ls_deduction=0.012095777075008172]   

In [21]:
print("desired_subset_size:", subset_size, "actual subset size:", len(subset))

desired_subset_size: 283781 actual subset size: 283781


In [ ]:
np.save("raw-clip-cov-10pct.npy", subset)

score=1.0438493142062784: 100%|██████████| 283781/283781 [09:33<00:00, 443.07it/s, sort_count=1250, sim-selected=0.014297999441623688, new_cross_deduction: 0.04337480014620724, ls_deduction=0.012095777075008172]

In [ ]:
FILENAME = "clip-cov-10pct"

if FILENAME != "":
    full.iloc[subset].to_csv(f"/home/user/clip_subset/subsets/{FILENAME}")